In [5]:
from pykeen.predict import predict_triples,predict_all,predict_target
import torch
from KGE import split_data,KGEmbedding

model=torch.load("checkpoints/TransE.pkl")

kge=KGEmbedding(model_name="TransE")
triple_factor_data_train, triple_factor_data_val, triple_factor_data_test, triple_factor_data=kge.construct_triples(train_path="data/TrainingSet.txt",
                                                                                                                    valid_path="data/EvaluationSet.txt",
                                                                                                                    test_path="data/TestSet.txt")

pred = predict_target(
    model=model,
    head="hmdb_id:HMDB0000126",
    relation="has_pathway",
    triples_factory=triple_factor_data,
)

You're trying to map triples with 4935 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 4898 from 1616121 triples were filtered out
You're trying to map triples with 4899 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 4867 from 1616121 triples were filtered out


In [53]:
import json
with open("data/TransE/id_to_entity.json","r") as f:
    id_to_entity=json.load(f)
    
import csv
triples=[]
with open("data/triples.txt", newline='', encoding='utf-8') as f:
    all_triples = csv.reader(f, delimiter='\t')
    for line in all_triples:
        triples.append(line)


In [112]:
def find_tail(model,triple_factory,id_to_entity,triples,head="hmdb_id:HMDB0000126",relation="has_pathway",topk=100,tail_type=None,predict_unknown=True):

    pred = predict_target(
        model=model,
        head=head,
        relation=relation,
        triples_factory=triple_factory,
    )
    
    topk_tails_scores=pred.df.sort_values("score").tail(topk)
    topk_tails_scores["tail_id"]=topk_tails_scores["tail_id"].map(str).map(id_to_entity)
    topk_tails=set(topk_tails_scores["tail_id"].tolist())
    
    known=set([i[2] for i in triples if i[0]==head and i[1]==relation])
    
    if predict_unknown:
        difference_set=topk_tails-known
    else:
        difference_set=topk_tails
    
    tails_scores_dict=topk_tails_scores.set_index("tail_id")["score"].to_dict()
    
    output={}
    for i in difference_set:
        if not tail_type or i.split(":")[0]==tail_type:
            output[i]=tails_scores_dict[i]
    
    return sorted(output.items(), key=lambda x: x[1],reverse=True)
    

In [122]:
def find_head(model,triple_factory,id_to_entity,triples,tail="hmdb_id:HMDB0000126",relation="has_pathway",topk=100,head_type=None,predict_unknown=True):
    
    pred = predict_target(
        model=model,
        tail=tail,
        relation=relation,
        triples_factory=triple_factory,
    )
    
    topk_heads_scores=pred.df.sort_values("score").head(topk)
    topk_heads_scores["head_id"]=topk_heads_scores["head_id"].map(str).map(id_to_entity)
    topk_heads=set(topk_heads_scores["head_id"].tolist())
    
    known=set([i[2] for i in triples if i[0]==tail and i[1]==relation])
    
    if predict_unknown:
        difference_set=topk_heads-known
    else:
        difference_set=topk_heads
    
    
    heads_scores_dict=topk_heads_scores.set_index("head_id")["score"].to_dict()
    
    output={}
    for i in difference_set:
        if not head_type or i.split(":")[0]==head_type:
            output[i]=heads_scores_dict[i]
    
    return sorted(output.items(), key=lambda x: x[1],reverse=True)
    

In [113]:
find_tail(model,triple_factor_data,id_to_entity,triples,head="hmdb_id:HMDB0000001",relation="related_to_disease",tail_type="disease")

[('disease:Colorectal cancer', -5.507114410400391),
 ('disease:Ulcerative colitis', -5.536546230316162),
 ('disease:Missing teeth', -5.8547468185424805),
 ('disease:Epilepsy', -5.878190040588379),
 ('disease:Prostate cancer', -5.972429275512695),
 ('disease:Pancreatic cancer', -6.07377815246582),
 ('disease:Attachment loss', -6.0748443603515625),
 ('disease:Irritable bowel syndrome', -6.0913848876953125),
 ('disease:Meningitis', -6.113704681396484),
 ('disease:Schizophrenia', -6.125271797180176),
 ('disease:Very Long Chain Acyl-CoA Dehydrogenase Deficiency',
  -6.228644847869873),
 ("disease:Crohn's disease", -6.2457685470581055),
 ('disease:Lipoyltransferase 1 Deficiency', -6.2962188720703125),
 ('disease:Hydrocephalus', -6.32382869720459),
 ('disease:Head injury', -6.383418083190918),
 ('disease:Periodontal Probing Depth', -6.408458709716797),
 ('disease:Hypermethioninemia', -6.429693222045898),
 ('disease:Canavan disease', -6.461075305938721),
 ('disease:Autosomal  dominant polycyst

In [123]:
find_head(model,triple_factor_data,id_to_entity,triples,tail="hmdbp_id:HMDBP01620",relation="related_to",head_type="hmdb_id")

[('hmdb_id:HMDB0265847', -14.789738655090332),
 ('hmdb_id:HMDB0289898', -14.82168960571289),
 ('hmdb_id:HMDB0292084', -14.943880081176758)]